In [1]:
import ipyleaflet as lf
from importlib import reload
import shapely.geometry as sg
import datetime as dt
from collections import defaultdict
import time
import operator as op
import numpy as np
import pandas as pd
from itertools import islice

In [2]:
import busboy.map.map as mp
import busboy.constants as c
import busboy.util as u
import busboy.prediction as p
import busboy.database as db
import busboy.model as m
import busboy.apis as api
import busboy.util.notebooks as nb

reload(mp)
reload(c)
reload(u)
reload(p)
reload(db)
reload(m)
reload(api)
reload(nb)

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [3]:
themap.clear_layers()

NameError: name 'themap' is not defined

In [5]:
all_stops = db.stops()
stops_by_name = {s.name: s for s in all_stops}

In [6]:
timetables_220 = list(api.timetables("220", stops_by_name))

In [65]:
len(list(u.unique(v for t in timetables_220 for v in t.variants if v.route == "220")))

10

In [72]:
timetables_205 = list(api.timetables("205", stops_by_name))

In [7]:
timetable = u.first(timetables_220)
variant = u.first(timetable.optional().variants)
variant.bind(lambda v: u.first(v.stops))

Just(value=Stop(id=StopId(raw='7338653551722184832'), name='Ovens (Grange Road Terminus)', latitude=51.87648, longitude=-8.64687, number=246671))

In [73]:
for v in u.unique(v for t in timetables_205 for v in t.variants if v.route == "205"):
    plot(v.stops, 0.1)

In [57]:
t = timetables_220[-2]
plot(list(t.variants)[0].stops, 0.1)

In [55]:
def plot(stops, interval):
    time.sleep(4)
    themap.clear_layers()
    for s in stops:
        themap._add_marker(s.latitude, s.longitude, s.name)
        time.sleep(interval)

In [67]:
stops = c.stops_on_220

In [5]:
for s in stops:
    themap.map.add_layer(
        lf.Marker(
                location=s.lat_lon,
                draggable=False,
                title=s.name,
            )
    )

In [9]:
for r in rs:
    themap.map.remove_layer(r)

AttributeError: 'Polygon' object has no attribute 'model_id'

In [68]:
rs = p.route_sections(stops, 0.001)

for r in rs:
    themap.add_polygon(r.polygon)

In [13]:
entries = db.snapshots(d = dt.date(2019, 2, 19))

entries_by_trip = defaultdict(list)
for e in entries:
    entries_by_trip[e.trip].append(e)
    
entries_by_vehicle = defaultdict(list)
for e in entries:
    entries_by_vehicle[e.vehicle].append(e)

In [14]:
rbn = db.routes_by_name()
entries_220 = [e for e in entries if e.route == rbn["220"].id]

entries_220_by_vehicle = defaultdict(list)
for e in entries_220:
    entries_220_by_vehicle[e.vehicle].append(e)
    
entries_220_by_trip = defaultdict(list)
for e in entries_220:
    entries_220_by_trip[e.trip].append(e)

In [15]:
trips = list(map(op.itemgetter(0), sorted(((t, len(v)) for t, v in entries_by_trip.items()), key=op.itemgetter(1), reverse=True)))
vehicles = list(map(op.itemgetter(0), sorted(((t, len(v)) for t, v in entries_by_vehicle.items()), key=op.itemgetter(1), reverse=True)))
route_ids = {e.route for e in entries}

trips_220 = list(map(op.itemgetter(0), sorted(((t, len(v)) for t, v in entries_220_by_trip.items()), key=op.itemgetter(1), reverse=True)))
vehicles_220 = list(map(op.itemgetter(0), sorted(((t, len(v)) for t, v in entries_220_by_vehicle.items()), key=op.itemgetter(1), reverse=True)))


In [16]:
route_entries = [e for e in entries if e.route == m.RouteId(raw='7338652709907596009')]
nb.plot_snapshots(themap, route_entries, 0.1, 4)

AttributeError: module 'busboy.util.notebooks' has no attribute 'plot_snapshots'

In [ ]:
vehicles

In [7]:
[len(entries_by_vehicle[v]) for v in vehicles]

[220976,
 26726,
 26038,
 21534,
 21131,
 17992,
 16738,
 16096,
 16091,
 15946,
 15163,
 15131,
 14575,
 14553,
 14541,
 14352,
 14184,
 13782,
 13463,
 13104,
 12907,
 12763,
 12712,
 12694,
 12281,
 12255,
 12174,
 12156,
 11917,
 11703,
 11554,
 11238,
 11125,
 11100,
 11021,
 10966,
 10806,
 10792,
 10647,
 10628,
 10408,
 10167,
 9950,
 9513,
 9504,
 9491,
 9270,
 9089,
 9081,
 8997,
 8834,
 8552,
 8546,
 8261,
 8126,
 8105,
 8099,
 8082,
 8024,
 7906,
 7901,
 7665,
 7605,
 7593,
 7586,
 7526,
 7494,
 7467,
 7131,
 7070,
 6970,
 6925,
 6916,
 6890,
 6589,
 6581,
 6482,
 6443,
 6260,
 5933,
 5782,
 5576,
 5355,
 5181,
 4988,
 4913,
 4888,
 4245,
 4234,
 4057,
 4009,
 3844,
 3766,
 3763,
 3616,
 3588,
 3525,
 3448,
 3440,
 3375,
 3162,
 3158,
 3151,
 3017,
 2935,
 2868,
 2801,
 2737,
 2681,
 2660,
 2656,
 2644,
 2603,
 2488,
 2428,
 2389,
 2371,
 2367,
 2363,
 2290,
 2283,
 2253,
 2213,
 2145,
 2138,
 2061,
 2024,
 1939,
 1935,
 1917,
 1911,
 1855,
 1784,
 1725,
 1702,
 1664,
 1657

In [17]:
[(v, len(entries_220_by_vehicle[v])) for v in vehicles_220]

[(VehicleId(raw=None), 21976),
 (VehicleId(raw='7338674957838188973'), 9912),
 (VehicleId(raw='7338674957838188879'), 8714),
 (VehicleId(raw='7338674957838188927'), 8653),
 (VehicleId(raw='7338674957838188926'), 8366),
 (VehicleId(raw='7338674957838189412'), 8316),
 (VehicleId(raw='7338674957838189204'), 8160),
 (VehicleId(raw='7338674957838188763'), 7291),
 (VehicleId(raw='7338674957838189389'), 7289),
 (VehicleId(raw='7338674957838189370'), 7272),
 (VehicleId(raw='7338674957838189394'), 7208),
 (VehicleId(raw='7338674957838189374'), 6264),
 (VehicleId(raw='7338674957838189398'), 4851),
 (VehicleId(raw='7338674957838188968'), 3605),
 (VehicleId(raw='7338674957838188807'), 3055),
 (VehicleId(raw='7338674957838188917'), 2569),
 (VehicleId(raw='7338674957838189431'), 2265),
 (VehicleId(raw='7338674957838188804'), 2064),
 (VehicleId(raw='7338674957838188805'), 2013),
 (VehicleId(raw='7338674957838188844'), 1319),
 (VehicleId(raw='7338674957838188866'), 924),
 (VehicleId(raw='7338674957838

In [11]:
es = list(islice(sorted(entries_220_by_vehicle[vehicles_220[2]], key=lambda e: e.poll_time), 500, 1000))
locations = lambda: ((e.latitude, e.longitude) for e in es)
markers = lambda: (lf.Marker(location=(e.latitude, e.longitude), draggable=False, title=e.poll_time.time().isoformat()) for e in es)
nb.plot_entries(themap, es, 0.1, 4)

In [86]:
line1 = lf.Polyline(locations=list(locations()))
themap.map.add_layer(line1)

In [81]:
cluster1 = lf.MarkerCluster(markers=list(markers())
themap.map.add_layer(cluster1)

In [88]:
group1 = lf.LayerGroup()
for e in es:
    group1.add_layer(lf.Marker(location=(e.latitude, e.longitude), draggable=False, title=e.poll_time.time().isoformat()))
themap.map.add_layer(group1)

In [76]:
control = lf.LayersControl(base_layers=[], overlays={"group1": group1, "group2": group2})
themap.map.add_control(control)

In [71]:
themap.map.remove_control(control)

In [3]:
themap = mp.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …